In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np

Matplotlib is building the font cache; this may take a moment.


In [18]:
df = pd.read_csv('data/Business_Dataset.csv')

def sort_group(group):
    group = group[(group['Year'] > 1995) & ~np.isin(group["Year"], [2020, 2021, 2024])]
    group = group.sort_values(by=['Year', 'quarter'])
    return group

inbounds = df.groupby('airport_2', group_keys=False).apply(sort_group)
outbounds = df.groupby('airport_1', group_keys=False).apply(sort_group)

inbounds = inbounds.groupby('airport_1')
for key, item in inbounds:
    print(inbounds.get_group(key), "\n\n")

/var/folders/5c/qd01_yk1187_jnr7n7jjw5m00000gn/T/ipykernel_10934/3622479328.py:1: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Business_Dataset.csv')
/var/folders/5c/qd01_yk1187_jnr7n7jjw5m00000gn/T/ipykernel_10934/3622479328.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  inbounds = df.groupby('airport_2', group_keys=False).apply(sort_group)


<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


/var/folders/5c/qd01_yk1187_jnr7n7jjw5m00000gn/T/ipykernel_10934/3622479328.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  outbounds = df.groupby('airport_1', group_keys=False).apply(sort_group)


In [ ]:
data['date'] = pd.to_datetime(data['Year'].astype(str) + 'Q' + data['quarter'].astype(str))
data.set_index('date', inplace=True)

features = ['nsmiles', 'fare', 'large_ms', 'lf_ms']
target = 'passengers'

X = data[features]
y = data[target]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')

    plt.plot(y_test.index, y_test, label='Actual')
    plt.plot(y_test.index, y_pred, label='Predicted', linestyle='--')
    plt.legend()
    plt.show()
